In [1]:
import import_ipynb
from milestone2 import *

from scipy.interpolate import CubicSpline
from scipy.integrate import solve_ivp
from scipy.interpolate import RectBivariateSpline

#from numpy import f2py

#with open('C:/Users/Alexander/Documents/College/Grad School Semester 4/Cosmo 2/Milestones - Python/bs_mod.f90') as bs_source:
#    bs_mod_file = bs_source.read()
#f2py.compile(bs_mod_file, modulename = "bs_mod")

#with open('C:/Users/Alexander/Documents/College/Grad School Semester 4/Cosmo 2/Milestones - Python/ode_solver.f90') as ode_source:
#    ode_solver_file = ode_source.read()
#f2py.compile(ode_solver_file, modulename = "ode_solver", verbose = 0)

#import rh_mod
#import healpix_types
#import ode_solver as FORT_ode

# Constants
ainit = 1.0e-8
xinit = math.log(ainit)
xend = np.log(1.0)
ketamax = 3000.0
#kmin = .1*H0/c
kmin = 1.0/η_spline(0.0)
kmax = ketamax/η_spline(0.0)
#kmax = 1.e3*H0/c
nk = 100
lmaxint = 6
npar = 6 + lmaxint


importing Jupyter notebook from milestone2.ipynb
importing Jupyter notebook from milestone1.ipynb
The surface of last scattering occured at x_* = -6.98 and z_* = 1068.80.

The half-way point of recombination occured at x_r = -7.16 and z_r = 1291.14.

Using only the Saha equation, the half-way point of recombination occured at x_r = -7.23 and z_r = 1380.38.



In [2]:
# Returns the time at which tight coupling ends.
# Either when (dτ < 10) OR (c*k/(Hp*dt) > .1) OR (x > x(recomb start))
def getTCtime(k):
    #n = 1000
    xstartrec = -math.log(1.0 + 1630.4)
    x = xinit
    
    
    while x < xstartrec:
        x = x + 1.0e-5
        hp = getHPrime(x)
        dτ = get_funct_der(x, τspl, 1)
        
        eps = c*k/(hp*dτ)
        
        if (abs(dτ) < 10.0 or abs(eps) > .1 or x >= xstartrec):
            xtc = x
            break
        
    '''
    for i in range(n):
        x = x + (xstartrec - xinit)/n
        
        #get_funct_der(x-array, spline, derivative order) 
        #if ( (get_funct_der(x, τspl, 1) < 10.0 and c*k/(getHPrime(x)*get_funct_der(x, τspl, 1)) > 0) or x > xstartrec ):
        if (abs(k/(getHPrime(x)*get_funct_der(x, τspl, 1)) >= 1.0/10.0) and (abs(get_funct_der(x, τspl, 1)) <= 10.0)) or x >= xstartrec:
            TCtime = x
            print(TCtime)
            print(i)
            break
    '''
    return xtc

In [3]:
def getdhp(x):
    xstep = 1.0e-3
    dhp = (getHPrime(x + xstep) - getHPrime(x - xstep))/ (2.0*xstep)
    
    return dhp

def yTCderivs(x, t):
    k0 = kcurrent
    #print('x:', x)
    #print('Hp:', getHPrime(x))
    hp = getHPrime(x)
    dτ = get_funct_der(x, τspl, 1)
    R = 4.0*Ωr/(3.0*Ωb*math.exp(x))
    #Θ2 = -20.0*c*k0*dydx[6]/(45.0*hp*dτ)
    Θ2 = -20.0*c*k0*y[6]/(45.0*hp*dτ)
    ψ = -y[4] - 12.0*H0**2.0*Ωr*Θ2/(c**2.0*k0**2.0*math.exp(2.0*x))
    #getdhp = getdHPrime(x)
    dydx = np.zeros(7)
    
    # δ    = [0]
    # δb   = [1]
    # v    = [2]
    # vb   = [3]
    # Φ    = [4]
    # θ0   = [5]
    # θ1   = [6]
    
    # dΦ
    dydx[4] = ψ - c**2.0*k0**2.0*y[4]/(3.0*hp**2.0) + H0**2.0*(Ωm*math.exp(-x)*y[0] + Ωb*math.exp(-x)*y[1] + 4.0*Ωr*math.exp(-2.0*x)*y[5])/(2.0*hp**2.0)
        
    # dδ
    dydx[0] = c*k0*y[2]/hp - 3.0*dydx[4]
        
    # dδb
    dydx[1] = c*k0*y[3]/hp - 3.0*dydx[4]
        
    # dv
    dydx[2] = -y[2] - c*k0*ψ/hp
    
    # dθ0
    dydx[5] = -c*k0*y[6]/hp - dydx[4]
        
    q = (-((1.0-2.0*R)*dτ + (1.0+R)*get_funct_der(x, τspl, 2))*(3.0*y[6] + y[3]) - c*k0*ψ/hp + (1.0 - getdhp(x)/hp)*c*k0*(-y[5] + 2.0*Θ2)/hp - c*k0*dydx[5]/hp)/((1.0 + R)*dτ + getdhp(x)/hp - 1.0)
        
    # dvb
    dydx[3] = (-y[3] - c*k0*ψ/hp + R*(q + c*k0*(-y[5] + 2.0*Θ2)/hp - c*k0*ψ/hp))/(1.0 + R)
    
    #dθ1
    dydx[6] = (q - dydx[3])/3.0
    
    return dydx


In [ ]:
def yderivs(x, t):
    #global dydx
    dydx = np.zeros(npar)
    k0 = kcurrent
    hp = getHPrime(t)
    dτ = get_funct_der(x, τspl, 1)
    R = 4.0*Ωr/(3.0*Ωb*math.exp(x))
    ψ = -y[4] - 12.0*H0**2.0*Ωr*y[7]/(c**2.0*k0**2.0*math.exp(2.0*x))

    # δ  = [0]
    # δb = [1]
    # v  = [2]
    # vb = [3]
    # Φ  = [4]
    # θ0 = [5]
    # θ1 = [6]

    # dΦ
    dydx[4] = ψ - c**2.0*k0**2.0*y[4]/(3.0*hp**2.0) + H0**2.0*(Ωm*math.exp(-x)*y[0] + Ωb*math.exp(-x)*y[1] + 4.0*Ωr*math.exp(-2.0*x)*y[5])/(2.0*hp**2.0)
    
    # dδ
    dydx[0] = c*k0*y[2]/hp - 3.0*dydx[4]

    # dδb
    dydx[1] = c*k0*y[3]/hp - 3.0*dydx[4]

    # dvb
    dydx[3] = -y[3] - c*k0*ψ/hp + dτ*R*(3.0*y[6] + y[3])

    # dv
    dydx[2] = -y[2] - c*k0*ψ/hp

    # dθ0
    dydx[5] = -c*k0*y[6]/hp - dydx[4]

    # dθ1
    dydx[6] = c*k0*y[5]/(3.0*hp) - 2.0*c*k0*y[7]/(3.0*hp) + c*k0*ψ/(3.0*hp) + dτ*(y[6] + y[3]/3.0)
    
    for l in range(2, lmaxint):
        dydx[5 + l] = l*c*k0*y[5 + l - 1]/((2.0*l + 1.0)*hp) - (l + 1.0)*c*k0*y[5 + l + 1]/((2.0*l + 1.0)*hp) + dτ*(y[5 + l] - y[5 + l]/10.0)
    #MISSING δ IN LAST TERM

    dydx[npar - 1] = c*k0*y[npar - 2]/hp - c*(npar - 1.0 + 1.0)*y[11]/(hp*getη(x)) + dτ*y[11]
    
    return dydx


In [4]:
# Initialize and solve the Boltzmann and Einstein equations
#Θ   = [[ [0 for q in range(ntotal)] for u in range(lmaxint)] for i in range(nk)]
#dΘ  = [[ [0 for q in range(ntotal)] for u in range(lmaxint)] for i in range(nk)]
ntotal = CONFORMAL_N
Θ = np.zeros(shape = (ntotal, lmaxint, nk))
dΘ = np.zeros(shape = (ntotal, lmaxint, nk))

#δ   = [[0 for col in range(nk)] for row in range(ntotal)]
δ = np.zeros(shape = (ntotal, nk))

#δb  = [[0 for col in range(ntotal)] for row in range(nk)]
δb = np.zeros(shape = (ntotal, nk))

#v   = [[0 for col in range(ntotal)] for row in range(nk)]
#vb  = [[0 for col in range(ntotal)] for row in range(nk)]
#dvb = [[0 for col in range(ntotal)] for row in range(nk)]

v = np.zeros(shape = (ntotal, nk))
vb = np.zeros(shape = (ntotal, nk))
dvb = np.zeros(shape = (ntotal, nk))

#Φ   = [[0 for col in range(ntotal)] for row in range(nk)]
#dΦ  = [[0 for col in range(ntotal)] for row in range(nk)]

Φ = np.zeros(shape = (ntotal, nk))
dΦ = np.zeros(shape = (ntotal, nk))

#Ψ   = [[0 for col in range(ntotal)] for row in range(nk)]
#dΨ  = [[0 for col in range(ntotal)] for row in range(nk)]

Ψ = np.zeros(shape = (ntotal, nk))
dΨ = np.zeros(shape = (ntotal, nk))

# Initialize k-grid ks
ks = [kmin + (kmax - kmin)*(((i - 1.0)/(nk - 1.0))**2.0) for i in range(nk)]

# Set up initial conditions for Boltzmann and Einstin equations
for i in range(nk):
    Φ[0][i] = 1.0
    δ[0][i] = 1.5
    δb[0][i] = 1.5

for i in range(nk):
    v[0][i] = c*ks[i]*Φ[0][i]/(2.0*getHPrime(xinit))
    vb[0][i] = v[0][i]
    Θ[0][0][i] = .5*Φ[0][i]
    Θ[0][1][i] = -c*ks[i]*Φ[0][i]/(6.0*getHPrime(xinit))
    Θ[0][2][i] = -20.0*c*ks[i]*Θ[0][1][i]/(45.0*getHPrime(xinit)*get_funct_der(xinit, τspl, 1))
    
    for l in range(3, lmaxint): ###3 or 2, lmaxint
        Θ[0][l][i] = -l*c*ks[i]*Θ[0][l - 1][i]/( (2.0*l + 1.0)*getHPrime(xinit)*get_funct_der(xinit, τspl, 1) )
        #get_funct_der(a, b, der) requires (x-array, spline, derivative order)


In [ ]:
# Integrate Perturbation Equations
y = np.zeros(npar)
#dydx = np.zeros(npar)
ytc = np.zeros(7) #y tight coupling - ytc


#Propogate each k-mode independently
for k in range(4):
    print('------------------------------')
    print('k:', k)
    kcurrent = ks[k]

    #Initialize equation set for tight coupling
    ytc[0] = δ[0][k]
    ytc[1] = δb[0][k]
    ytc[2] = v[0][k]
    ytc[3] = vb[0][k]
    ytc[4] = Φ[0][k]
    ytc[5] = Θ[0][0][k]
    ytc[6] = Θ[0][1][k]
    
    #Find the time at which tight couplint (TC) is assumed
    #and integrate equations to that time
    xtc = getTCtime(kcurrent)
    ntight = 500
    xtight = np.linspace(xinit, xtc, ntight)# endpoint=True)
    x_full = np.linspace(xtc, xend, CONFORMAL_N - ntight, endpoint=True)
    #Integrate from xinit until the end of TC using the TC equations 
    
    #print('xinit:', xinit)
    #print('xtc:', xtc)
    #print('Hp(xtc):', getHPrime(xtc))
    print('before')
    print('////////////////////////////////////////////////////////////')
    #print('xtight:', xtight)
    print(ytc)
    print('////////////////////////////////////////////////////////////')
    
    sol_tight = solve_ivp(yTCderivs, [xinit, xtc], ytc, t_eval=xtight, rtol=1e-6, atol=1e-6)
    
    y_after_tc = sol_tight.y[:, -1]
    
    #Set up variables for integration from the end of tight coupling until today
    for i in range(7): y[i] = y_after_tc[i]
    y[7] = -20.0*c*kcurrent*y[6]/(45.0*getHPrime(xtc)*get_funct_der(xtc, τspl, 1))
    
    for l in range(2, lmaxint):
        y[6 + l] = -l*c*kcurrent*y[6 + l - 1]/((2.0*l + 1)*getHPrime(xtc)*get_funct_der(xtc, τspl, 1))
    
    
    
    for i in range(ntotal):
        
        if (i==1 and (x_grid[i] != xtc)):
            sol_full = solve_ivp(yderivs, [xtc, xend], y, t_eval=x_full, rtol=1e-6, atol=1e-6)
        else:
            sol_full = solve_ivp(yderivs, [xtc, xend], y, t_eval=x_full, rtol=1e-6, atol=1e-6)
        
        
        
        
        
        δ[i][k] = sol_full[0]
        δb[i][k] = sol_full[1]
        v[i][k] = sol_full[2]
        vb[i][k] = sol_full[3]
        Φ[i][k] = sol_full[4]
        Θ[i][0][k] = sol_full[5]
        Θ[i][1][k] = sol_full[6]
        Θ[i][2][k] = -20.0*c*kcurrent*Θ[i][1][k]/(45.0*getHPrime(x_grid[i])*get_funct_der(x_grid[i], τspl, 1))
                                                  
        for l in range(3, lmaxint):
            Θ[i][l][k] = sol_full[5 + l]
            
        Ψ[i][k] = -y[4] - 12.0*H0**2.0*Ωr*y[7]/(c**2.0*kcurrent**2.0*math.exp(x_grid[i])**2.0)
        
        
        
        
        
        
        if x_grid[i] >= xtc:
            for i in range(7): y[i] = ytc[i]
                
            y[7] = -20.0*c*kcurrent*y[6]/(45.0*getHPrime(xtc)*get_funct_der(xtc, τspl, 1))
            
            for l in range(2, lmaxint):
                y[6 + l] = -l*c*kcurrent*y[6 + l - 1]/((2.0*l + 1)*getHPrime(xtc)*get_funct_der(xtc, τspl, 1))
                
            break

In [21]:
print(sol_tight.y[:, -1])

[ 1.50000000e+00  1.50000000e+00  4.73839087e-04  4.73839087e-04
  1.00000000e+00  5.00000000e-01 -1.57946362e-04]


In [ ]:
'''
    ###for i in range(1, len(ytc)):
    ###    rng  = np.linspace(xinit, xtc, 100)
    ###    dydx = yTCderivs(rng[i], ytc, dydx)
    ###    temp = bsint.bsintegrate(yderivs, dydx, rng[i - 1], rng[i + 1], args = (kcurrent,))
    ###    #temp = odeint(yderivs, dydx, rng[(i-1):i+1])
    ###    dydx = temp[1]
    
    #Set up variables for integration from the end of tight coupling until today
    for i in range(7): y[i] = ytc[i]
    y[7] = -20.0*c*kcurrent*y[6]/(45.0*getHPrime(xtc)*get_funct_der(xtc, τ2spl, 1))
    
    for o in range(3, lmaxint):
        y[5 + o] = -o*c*kcurrent*y[5 + o - 1]/((2.0*o + 1.0)*getHPrime(xtc)*get_funct_der(xtc, τ2spl, 1))

    for i in range(ntotal):#should be (ntotal)
        #Integrate equations from TC to today
        if ( (i == 0) and (xtc != xgrid[i]) ): #getting out of TC-mode
            TCrng = np.linspace(xtc, xgrid[i], 100)
            temp = odeint(yderivs, dydx, TCrng)
            dydx = temp[1]
            
        elif i > 0: #going onwards to today
            temp = odeint(yderivs, dydx, xgrid[(i-1):i+1])
            dydx = temp[1]
            
        #print('ylen: ', len(y))
        #print('   k: ', k)
        #print('   i: ', i)
        # Store variables at time step i in global variables
        δ[i][k] = y[0]
        δb[i][k] = y[1]
        v[i][k] = y[2]
        vb[i][k] = y[3]
        Φ[i][k] = y[4]
        
        for l in range(lmaxint): Θ[i][l][k] = y[5 + l]
        
        Ψ[i][k] = -y[4] - 12.0*H0**2.0*Ωr*y[7]/(c**2.0*kcurrent**2.0*(math.exp(xgrid[i]))**2.0)
        
        # Store derivatives that are required for C_l estimation
        #yderivs(xgrid[i], y, dydx)
        dΦ[i][k] = dydx[4]
        dvb[i][k] = dydx[3]
        for b in range(lmaxint): dΘ[i][b][k] = dydx[5:5+lmaxint]
        dΦ[i][k] = -dydx[4] - (12.0*H0**2.0*Ωr*math.exp(-1.0*xgrid[i])*(-2.0*y[7] + dydx[7])/(c**2.0*kcurrent**2.0))
    '''

In [ ]:
klist = [0, 11, 29, 39, 84, 99]

plt.figure()
plt.plot(xη, δ[0:len(xη-1)][0])


In [2]:
import numpy as np

xx = np.zeros((3, 5))

yy= np.zeros(shape = (3, 5))
print(xx)
print('--0-----------')
print(yy)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
--0-----------
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
